In [2]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import RZZGate
from qiskit_aer import AerSimulator

In [3]:
def apply_one_axis_twist(circuit, qubits, chi_t):
    n = len(qubits)
    for i in range(n):
        for j in range(i + 1, n):
            circuit.append(RZZGate(2 * chi_t), [qubits[i], qubits[j]])
    return circuit

In [4]:
def apply_dipole_interaction(circuit, qubits, chi_t):
    n = len(qubits)
    for i in range(n):
        for j in range(i + 1, n):
            qi, qj = qubits[i], qubits[j]

            # ZZ term
            circuit.append(RZZGate(2 * chi_t), [qi, qj])

            # XX term 
            theta_x = -chi_t
            circuit.h(qi); circuit.h(qj)
            circuit.cx(qi, qj)
            circuit.rz(theta_x, qj)
            circuit.cx(qi, qj)
            circuit.h(qi); circuit.h(qj)

            # YY term
            theta_y = -chi_t
            circuit.rx(np.pi/2, qi); circuit.rx(np.pi/2, qj)
            circuit.cx(qi, qj)
            circuit.rz(theta_y, qj)
            circuit.cx(qi, qj)
            circuit.rx(-np.pi/2, qi); circuit.rx(-np.pi/2, qj)
            
    return circuit

In [5]:
def calculate_jz_and_delta_jz(counts, n_qubits=3):
    expectation = 0
    expectation_sq = 0
    total_shots = sum(counts.values())
    
    for bitstring, count in counts.items():
        n_zeros = bitstring.count('0')  
        n_ones = bitstring.count('1')   
        jz_value = (n_zeros - n_ones) / 2
        prob = count / total_shots
        expectation += prob * jz_value
        expectation_sq += prob * (jz_value ** 2)
    
    delta_jz = (expectation_sq - expectation ** 2) ** 0.5
    return expectation, delta_jz

In [6]:
# Parameters
N = 3                    
shots = 1000
backend = AerSimulator()
tau_max_ns = 650.0                          
tau_points = 70
tau_grid_s = np.linspace(0.0, tau_max_ns*1e-9, tau_points)
d = 1e6
chi_oat_hz  = 1e6
theta_points = 181
theta_grid   = np.linspace(0, 2*np.pi, theta_points)

In [7]:
def rotate_for_theta(qc, theta):
    for q in range(N):
        qc.rx(-theta, q)

In [8]:
def delta_J_theta_for_H(hamiltonian: str, tau_s: float, theta: float) -> float:
    if hamiltonian.upper() == "DH":
        chi_t = 2*np.pi * d * tau_s
        apply_func = apply_dipole_interaction
    elif hamiltonian.upper() == "OAT":
        chi_t = 2*np.pi * chi_oat_hz * tau_s
        apply_func = apply_one_axis_twist
    else:
        raise ValueError("hamiltonian must be 'OAT' or 'DH'")

    qc = QuantumCircuit(N)
    qc.h(range(N))                                     
    apply_func(qc, list(range(N)), chi_t)              
    rotate_for_theta(qc, theta)                        
    qc.measure_all()

    counts = backend.run(transpile(qc, backend), shots=shots).result().get_counts()
    _, dJ = calculate_jz_and_delta_jz(counts, n_qubits=N)   
    return float(dJ) 

In [9]:
def find_tau_min(hamiltonian: str):
    best_sigma = np.inf
    best_tau   = None
    best_theta = None
    for tau in tau_grid_s:
        sigmas_tau = [delta_J_theta_for_H(hamiltonian, tau, th) for th in theta_grid]
        s_tau = np.min(sigmas_tau)
        if s_tau < best_sigma:
            best_sigma = s_tau
            best_tau   = tau
            best_theta = theta_grid[int(np.argmin(sigmas_tau))]
    return best_tau, best_sigma, best_theta

In [ ]:
# τ_min for OAT
tau_min_OAT, sigma_min_OAT, theta_min_OAT = find_tau_min("OAT")
tau_min_DH,  sigma_min_DH,  theta_min_DH  = find_tau_min("DH")

print(f"[OAT] τ_min = {tau_min_OAT*1e9:7.2f} ns, σ_min = {sigma_min_OAT:.5f}, at θ ≈ {np.degrees(theta_min_OAT):.1f}°")
print(f"[DH ] τ_min = {tau_min_DH*1e9:7.2f} ns, σ_min = {sigma_min_DH :.5f}, at θ ≈ {np.degrees(theta_min_DH ):.1f}°")

# Ellipse curves at their τ_min
sigma_theta_OAT = np.array([delta_J_theta_for_H("OAT", tau_min_OAT, th) for th in theta_grid])
sigma_theta_DH  = np.array([delta_J_theta_for_H("DH",  tau_min_DH,  th) for th in theta_grid])

In [ ]:
sigma0 = np.sqrt(N)/2.0
sigma_theta_OAT_norm = sigma_theta_OAT / sigma0
sigma_theta_DH_norm  = sigma_theta_DH  / sigma0

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10, 4.2), subplot_kw={"projection": "polar"}, sharex=True, sharey=True)

# OAT
axes[0].plot(theta_grid, sigma_theta_OAT_norm)
axes[0].set_title("Uncertainty Ellipse — OAT")
axes[0].set_rticks([0.5, 1.0, 1.5])
axes[0].grid(True)

# DH
axes[1].plot(theta_grid, sigma_theta_DH_norm)
axes[1].set_title(r"Uncertainty Ellipse — Dipole ( $d/(2\pi)=1\,\mathrm{MHz}$ )")
axes[1].set_rticks([0.5, 1.0, 1.5])
axes[1].grid(True)

fig.suptitle(fr"Squeezing Ellipses at $\tau_{{\min}}$ (N={N}); normalized to $\sigma_0=\sqrt{{N}}/2$", y=1.04)
plt.tight_layout()
plt.show()